In [ ]:
!pip install -q bitsandbytes peft trl accelerate datasets transformers pandas

In [ ]:
import os
import torch
import transformers
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import pandas as pd

In [ ]:
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

In [ ]:
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token=os.environ['HF_TOKEN']
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"":0},
    token=os.environ['HF_TOKEN']
)

In [ ]:
text = "What is the summary about this Rwandan Law Nº68/2018 of 30/08/2018?"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
os.environ["WANDB_DISABLED"] = "false"

In [ ]:
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM"
)

In [ ]:
from datasets import load_dataset

data = load_dataset("iradukunda-dev/offences_and_penalties_in_general_2018_dataset")
data = data.map(lambda samples: tokenizer(samples["input"]), batched=True)


In [ ]:
data['train']['instruction']

In [ ]:
def formatting_func(example):
  text = f"Instruction: {example['instruction'][0]}\nInput: {example['input'][0]}\nOutput: {example['output'][0]}"
  return [text]

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=False,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",

  ),
    peft_config=lora_config,
    formatting_func=formatting_func
)

In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained("finetuned-DeepSeek-R1-Distill-Qwen-7B")

In [ ]:
text = "What is the Article 152?"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))